<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/1_data_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

In [0]:
%cd "/gdrive/My Drive/tensor-flow-state/tensor-flow-state"

In [0]:
sensor_list = ["RWS01_MONIBAS_0021hrl0403ra.csv", "RWS01_MONIBAS_0021hrl0409ra.csv", "RWS01_MONIBAS_0021hrl0414ra.csv", "RWS01_MONIBAS_0021hrl0420ra.csv", "RWS01_MONIBAS_0021hrl0426ra.csv"]

In [0]:
import datetime
def dateparse (time_in_secs):
    # Unix/epoch time to "YYYY-MM-DD HH:MM:SS"
    return datetime.datetime.fromtimestamp(float(time_in_secs))

import pandas as pd
def repair_datetime_index(df, freq = "T"):
    df = df.loc[~df.index.duplicated(keep = "first")] # remove duplicate date time indexes
    df = df.reindex(pd.date_range(start = df.index.min(), end = df.index.max(), freq = freq)) # add missing date time indexes
    return df

import numpy as np
def fix_values(df):
    # The order of these operations is currently important! Pay attention when making changes
    df["speed_limit"] = np.where((df.index.hour < 19) & (df.index.hour >= 6), 100, 120)
    df.loc[df.flow < 0, "flow"] = np.nan # flow is either -2 (missing data) or 0 or positive. -2 to nan
    df.loc[df.speed < -1, "speed"] = np.nan # -2 (missing data) as well as oddities (-1.33, an average over -2 and -1 lanes?) to nan 
    df.speed.mask(df.speed == -1, df.speed_limit, inplace = True) # -1 means no cars, setting it to speed limit
    df.loc[(df.speed < 0) & (df.speed > -1), "speed"] = 0 # anything else below zero is between 0 and -1, occuring when some lanes have non-moving cars while others have have no cars.
    df.speed.mask(df.speed > df.speed_limit, df.speed_limit, inplace = True) # cap speed at speed_limit, since higher speed dosn't add to representation
    return df

import os
def reduce_cols(sensors, path_in = "data/ndw_raw/", path_out = "data/"):
    for sensor in sensors:
        df = pd.read_csv(os.path.join(path_in, sensor), header = None, \
                         usecols = [0, 86, 87], names = ["timestamp", "speed", "flow"], \
                         index_col = "timestamp", parse_dates = True, date_parser = dateparse)
        df.flow /= 60 # change flow unit to min^-1
        df = repair_datetime_index(df)
        df = fix_values(df)        
        df.to_csv(path_out + sensor)


In [0]:
reduce_cols(sensor_list)